In [2]:
import numpy as np
import json

In [8]:
with open('train.json') as f:
    data = f.readlines()
    
data = [x.strip() for x in data]
data[0], len(data)

('{\'reviewTime\': \'09 26, 2013\', \'reviewText\': "The model in this picture has them rolled up at the top because they are actually very high waisted! that\'s my only complaint though, because they are very good quality, and fit really well! I am 5\'2&#34; 120lbs with thick thighs and i love them i can\'t wait to wear them out!", \'helpful\': {\'nHelpful\': 0, \'outOf\': 0}, \'reviewerID\': \'U490934656\', \'reviewHash\': \'R798569390\', \'categories\': [[\'Clothing, Shoes & Jewelry\', \'Women\'], [\'Clothing, Shoes & Jewelry\', \'Novelty, Costumes & More\', \'Novelty\', \'Clothing\', \'Women\', \'Leggings\']], \'unixReviewTime\': 1380153600, \'itemID\': \'I402344648\', \'rating\': 4.0, \'summary\': \'High Waisted\', \'categoryID\': 0}',
 200000)

In [12]:
json_string = json.loads(data[0])[0]

json_string
# for k, v in json_string.items():
#     print(k, v)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)